In [1]:
%%capture
!pip install python-dotenv
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install sentence_transformers

### **練習 #1**

使用 Extractive QA model 以 `pipeline` 做 question-answering 任務：
- **給定文本**：the name of repo is bert-base-uncased
- **問題目標**：問模型 repo 的名稱
- **預期答案**：bert-base-uncased

In [2]:
# TODO
# practice 1 不需要特別指定模型，pipeline 預設載入 distilbert-base-cased-distilled-squad,
# 其為 Extractive QA 類摘要模型

from transformers import pipeline

In [3]:
pipe = pipeline(task="question-answering")
pipe(context="the name of repo is bert-base-uncased",
     question="what is the repo name")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recom

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

{'score': 0.9398112893104553,
 'start': 20,
 'end': 37,
 'answer': 'bert-base-uncased'}

利用 Hugging Face 的 text-generation model 與 Sentence Transformers embedding model 實作 QA 檢索聊天機器人。</p>
基於提供的資料集，使用 Embedding Cosine Similarity 檢索參考資料，再透過 LLM 生成答案。</p>

1. Baseline
   - 將 demo 中的資料，替換成我們提供 or 自己的資料集
   - 能夠檢索相似資料
   - 基於檢索的資料進行回答
2. Advanced（Optional）
   - Embedding 怎麼儲存？每次都要重新計算嗎？
   - 該如何處理太久以前的歷史資料？
   - 利用 Gradio or Hugging Face Spaces 部署、分享 Chatbot

In [4]:
from pprint import pprint

from torch import cuda, bfloat16

from transformers import pipeline
from transformers import Conversation
conversation = Conversation() # 建立一個對話 Conversation 物件

from transformers import BitsAndBytesConfig, AutoConfig, AutoModelForCausalLM, AutoTokenizer
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)

from sentence_transformers import SentenceTransformer


embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")

cuda:0


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

因為載入模型較大，使用 T4 GPU 時建議進行量化，以下程式為量化處理過程，</p>
在此先不贅述，有興趣的可以參考 Hugging face 官方文件～</p>

與前面範例不同的是，模型載入方法，我們透過 `AuToModelForCausalLM` 實例化模型，將其作為參數傳入 `pipeline`。

In [5]:
model_id = 'MediaTek-Research/Breeze-7B-32k-Instruct-v1_0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model_config = AutoConfig.from_pretrained(
    model_id
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

hf_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/512M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [9]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# 載入語料庫
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/LLM讀書會/cakeresume.xlsx")
df.drop(columns=['Unnamed: 0','index'], inplace=True)

In [15]:
# 轉成json文字格式
qa_data = df.apply(lambda row: f'"""{row.to_json(force_ascii=False)}"""', axis=1).tolist()

In [47]:
chatbot = pipeline(
    "text-generation",
    model=hf_model,
    tokenizer=tokenizer, # Tokenizer，要與模型匹配，主要提供 chat 模式時的特殊符號
    max_new_tokens=1024, # 模型最多可以生成多少字
    return_full_text=False # 控制 pipeline 只輸出 AI Message
)

Demo

In [ ]:
# qa_data = [
#     """1. 入營注意事項：役男入營前無須先行理髮，待入營後統一安排理髮，再向役男收取費用
#     2. 役男可攜帶手機(不得為大陸廠牌，且不提供充電)，部隊會集中保管，於每日夜間以定時定點方式使用。
#     3. 營區不提供充電，建議攜帶拋棄式(手動)刮鬍刀；不可攜帶噴霧式液體。
#     4. 役男可配戴隱形眼鏡，但因基礎訓練期間生活緊湊，而隱形眼鏡需相當時間消毒清洗，建議配戴鏡片眼鏡為宜，並可多備一副眼鏡，以供替換。
#     """,

#     """報到入營時應該文件：
#     1. 徵集令。
#     2. 役男本人之國民身分證正本。
#     3. 私章。
#     4. 健保ＩＣ卡。
#     5. 郵局存摺正面影本。
#     6. 替代役役男輔導需求調查表。
#     7. 個人特殊醫療用品。
#     """,

#     """
#     替代役訓練天數
#     基礎訓練：21天(含撥交日)。(自253梯次起修正)
#     專業訓練：以各分發需用機關不同而有個不同期間的專業訓練。
#     """,

#     """
#     折抵役期規定
#     請備妥相關證明文件，如高中（職）以上各級學校經軍訓主管驗證加註折抵役期日數之成績單、大專集訓結訓證書、
#     驗退（停役）證明書或軍事學校退學（開除）證明書等正本（驗證後退還）。82年次以前出生者，
#     合計不得逾30日，83-93年次以後出生者，合計不得逾15日。已受軍事入伍訓練者，請於收到徵集令時，向戶籍地區公所提出免受基礎訓練申請。
#     """,

#     """
#     要當兵時，健保要辦理轉出轉入嗎?
#     即將入營服常備兵役之役男需持徵集令向全民健保加保單位辦理轉出，轉出日期填報「入營當月份」，轉入單位免填，後續將由國軍單位接續辦理後續轉入程序。
#     """,

#     """當兵期間義務役薪水的入帳戶一定要郵局嗎?
#     依現行規定目前有郵局、台新、土地銀行跟合作金庫這四家金融機構可使用。"""
# ]

In [17]:
import torch
import os

In [18]:
# 開GPU下去加速
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device is {device}')

device is cuda


In [53]:
%%time
# 把語料庫的向量先計算好

# 直接讀取：8.48 ms
# 計算 10  obs  / 2 sec,
# 計算 100  obs / 9.2 sec,
# 計算 1000 obs / 1min 16sec
# 計算 13,000 obs / 19 min

input_data = qa_data

file_path = '/content/drive/MyDrive/Colab Notebooks/LLM讀書會/embeddings.pt'

if os.path.exists(file_path):
    try:
        # 先確認embedding有沒有存在
        source_embeddings = torch.load(file_path)
        print("文件下載成功。")
        print(f"嵌入向量的形状: {source_embeddings.shape}")
    except Exception as e:
        print(f"加载文件時出錯: {e}")
else:
    print(f"文件 {file_path} 不存在。")
    embedding_model.to(device)
    source_embeddings = embedding_model.encode(input_data, convert_to_tensor=True).to(device)
    torch.save(source_embeddings, '/content/drive/MyDrive/Colab Notebooks/LLM讀書會/embeddings.pt')



文件下載成功。
嵌入向量的形状: torch.Size([13204, 1024])
CPU times: user 17 ms, sys: 65.2 ms, total: 82.2 ms
Wall time: 94.1 ms


In [20]:
from typing import List
import numpy as np

def get_answer(query: str, source: List[str], source_embeddings):

    # 先計算查詢向量
    query_embedding = embedding_model.encode(query)

    most_related_sentence = None
    max_similarity = 0

    # 計算相似度
    for sentence, sentence_embedding in zip(source, source_embeddings):
      sim = calculate_cosine_similarity(query_embedding, sentence_embedding)
      if sim > max_similarity:
          most_related_sentence = sentence
          max_similarity = sim

    return most_related_sentence

def calculate_cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [27]:
%%capture
# !pip install gradio
import gradio as gr

In [51]:
# Gardio 介面

gr_chatbot = gr.Chatbot()
conversation = Conversation()

def chatbot_response(user_query, history):

  # 計算語料庫相似度
  answer = get_answer(user_query, qa_data, source_embeddings.cpu())

  llm_input = f"""請你基於以下資訊，運用繁體中文回答使用者的問題，不要超過100個字：
  {answer}
  ===
  問題：{user_query}
  """

  # 將 conversation.messages 丟給 chatbot
  conversation.add_user_input(llm_input)
  chatbot_result = chatbot(conversation.messages)[0]['generated_text']
  conversation.append_response(chatbot_result)

  return chatbot_result

# 使用Gradio建立介面
iface = gr.ChatInterface(
    fn=chatbot_response,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a yes or no question", container=False, scale=7),
    title="CakeResumn Chatbot",
    description="來吧，年薪三百萬的工作跟秘訣我都放在這裡了，來問我吧！",
    theme="soft",
    cache_examples=True,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear"
)

# 運行應用
app = iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3622c64f8bb75f68dd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


---

In [52]:
# user_query = input(">>")
# conversation = Conversation()

# while user_query.lower() != "bye":
#     print(f"user: {user_query}")
#     # 尋找最相似的文件
#     answer = get_answer(user_query, qa_data, source_embeddings.cpu())
#     llm_input = f"""請你基於以下資訊回答使用者的問題，不要超過100個字
#     {answer}
#     ===
#     問題：{user_query}
#     """
#     conversation.add_user_input(llm_input)
#     # 將 conversation.messages 丟給 chatbot
#     chatbot_result = chatbot(conversation.messages)[0]['generated_text']

#     print(f"AI: {chatbot_result}")
#     conversation.append_response(chatbot_result)

#     print(f"\n reference answer : {answer}")
#     user_query = input(">>>")
